Build a model that can rate the sentiment of a Tweet based on its content.

You'll build an NLP model to analyze Twitter sentiment about Apple and Google products. The dataset comes from CrowdFlower via data.world. Human raters rated the sentiment in over 9,000 Tweets as positive, negative, or neither.

Aim for a Proof of Concept There are many approaches to NLP problems - start with something simple and iterate from there. For example, you could start by limiting your analysis to positive and negative Tweets only, allowing you to build a binary classifier. Then you could add in the neutral Tweets to build out a multiclass classifier. You may also consider using some of the more advanced NLP methods in the Mod 4 Appendix.

Evaluation Evaluating multiclass classifiers can be trickier than binary classifiers because there are multiple ways to mis-classify an observation, and some errors are more problematic than others. Use the business problem that your NLP project sets out to solve to inform your choice of evaluation metrics.

Data: https://data.world/crowdflower/brands-and-product-emotions

# Business Understanding

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# nltk related imports
import nltk
from nltk.tokenize import RegexpTokenizer, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [3]:
!ls ../../data

judge-1377884607_tweet_product_company.csv


In [4]:
df = pd.read_csv('../../data/judge-1377884607_tweet_product_company.csv', encoding = 'unicode_escape')
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [6]:
first_tweet = df['tweet_text'][0]
first_tweet

'.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.'

In [7]:
#lowercase
first_tweet_lower = first_tweet.lower()
first_tweet_lower

'.@wesley83 i have a 3g iphone. after 3 hrs tweeting at #rise_austin, it was dead!  i need to upgrade. plugin stations at #sxsw.'

In [8]:
#tweet tokenizer
tweet_tknzr = TweetTokenizer(strip_handles=True)
first_tweet_lower_tt = tweet_tknzr.tokenize(first_tweet_lower)
first_tweet_lower_tt

['.',
 'i',
 'have',
 'a',
 '3g',
 'iphone',
 '.',
 'after',
 '3',
 'hrs',
 'tweeting',
 'at',
 '#rise_austin',
 ',',
 'it',
 'was',
 'dead',
 '!',
 'i',
 'need',
 'to',
 'upgrade',
 '.',
 'plugin',
 'stations',
 'at',
 '#sxsw',
 '.']

In [9]:
#turn tokenized words back into tweet
first_tweet_lower_tweet = " ".join(first_tweet_lower_tt)
first_tweet_lower_tweet

'. i have a 3g iphone . after 3 hrs tweeting at #rise_austin , it was dead ! i need to upgrade . plugin stations at #sxsw .'

In [10]:
#use regexptokenizer
pattern = r"(?u)\w{2,}" # select all words with 2 or more characters
         #r"(?u)\b\w\w+\b"
         #r'\w+'     <-REMOVES PUNCTUATION
regexp_tknzr = RegexpTokenizer(pattern)
first_tweet_regexp = regexp_tknzr.tokenize(first_tweet_lower_tweet)
first_tweet_regexp

['have',
 '3g',
 'iphone',
 'after',
 'hrs',
 'tweeting',
 'at',
 'rise_austin',
 'it',
 'was',
 'dead',
 'need',
 'to',
 'upgrade',
 'plugin',
 'stations',
 'at',
 'sxsw']

In [11]:
# create list of stopwords in English
stopwords_list = stopwords.words('english')

#remove stopwords
first_tweet_sw_removed = [word for word in first_tweet_regexp if word not in stopwords_list]
first_tweet_sw_removed

['3g',
 'iphone',
 'hrs',
 'tweeting',
 'rise_austin',
 'dead',
 'need',
 'upgrade',
 'plugin',
 'stations',
 'sxsw']

In [12]:
# create lemma object
lemma = WordNetLemmatizer()
first_tweet_lemma = [lemma.lemmatize(token) for token in first_tweet_sw_removed]
first_tweet_lemma

['3g',
 'iphone',
 'hr',
 'tweeting',
 'rise_austin',
 'dead',
 'need',
 'upgrade',
 'plugin',
 'station',
 'sxsw']

# *Doing 2 Train_Test_Splits first to avoid Data Leakage.<br> Then Preprocessing each split*

In [13]:
df = df.rename(columns = {'tweet_text': 'Tweet', 
                         'emotion_in_tweet_is_directed_at': 'Product', 
                         'is_there_an_emotion_directed_at_a_brand_or_product': 'Sentiment'})
df.head() #Sanity Check

,Tweet,Product,Sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [14]:
df.shape

(9093, 3)

In [15]:
df['Tweet'].iloc[9092]

'\x8cÏ¡\x8eÏà\x8aü_\x8b\x81Ê\x8b\x81Î\x8b\x81Ò\x8b\x81£\x8b\x81Á\x8bââ\x8b\x81_\x8b\x81£\x8b\x81\x8f\x8bâ_\x8bÛâRT @mention Google Tests \x89ÛÏCheck-in Offers\x89Û\x9d At #SXSW {link}'

In [16]:
df['Tweet'].iloc[6]

nan

In [17]:
df.drop([6, 9092], inplace=True)
df.drop_duplicates(inplace=True)
df['Tweet'].dropna(inplace=True)

In [18]:
df.isna().sum()

Tweet           0
Product      5787
Sentiment       0
dtype: int64

In [19]:
df['Sentiment'].value_counts()

No emotion toward brand or product    5374
Positive emotion                      2970
Negative emotion                       569
I can't tell                           156
Name: Sentiment, dtype: int64

In [20]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == "Positive emotion" else 0)

In [21]:
X = df[['Tweet']]
y = df['Sentiment']
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=0.25, random_state=42)

In [22]:
#BASELINE UNDERSTANDING
y_train.value_counts(normalize=True)

0    0.67636
1    0.32364
Name: Sentiment, dtype: float64

In [23]:
# #If we did a multi class
# dict_sent = {'No emotion toward brand or product':1, 
#              'Positive emotion':2,
#              'Negative emotion':0,
#              "I can't tell": 1}
# df['Sentiment'] = df['Sentiment'].map(dict_sent)

In [24]:
# #Preprocess targets
# y_train = y_train.apply(lambda x: 1 if x == "Positive emotion" else 0)
# y_val = y_val.apply(lambda x: 1 if x == "Positive emotion" else 0)
# y_test = y_test.apply(lambda x: 1 if x == "Positive emotion" else 0)

In [25]:
X_train.head()

,Tweet
2324,@mention Can we make you an iPhone case with T...
5632,RT @mention Come party down with @mention &amp...
1751,#winning #winning - just gave away 5 red mophi...
5799,RT @mention google &amp; facebook have an offi...
3339,Rumor of Google launching their new social net...


In [26]:
X_train.shape

(6121, 1)

In [27]:
X_val.shape

(2041, 1)

In [28]:
#Instantiate necessary tools
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stopwords_list = stopwords.words("english")
lemma = WordNetLemmatizer()
tweet_tknzr = TweetTokenizer(strip_handles=True)

In [29]:
def clean_tweets(text):
    no_handle = tweet_tknzr.tokenize(text)
    tweet = " ".join(no_handle) 
    #remove http websites, pound sign, any words in brackets, any words with ampersand right in front
        # ?, www dot com websites, links, videos, and non english characters
    #clean = re.sub("((^|\W)@\b([-a-zA-Z0-9._]{3,25})\b) \
        #|(&[a-z]+;)|([^\w\s]) \
    clean = re.sub("(https?:\/\/\S+) \
                   |(#[A-Za-z0-9_]+) \
                   |(\{([a-zA-Z].+)\}) \
                   |(&[a-z]+;) \
                   |(www\.[a-z]?\.?(com)+|[a-z]+\.(com))\
                   |({link})\
                   |(\[video\])\
                   |([^\x00-\x7F]+\ *(?:[^\x00-\x7F]| )*)"," ", tweet)
    lower = clean.lower()
    token_list = tokenizer.tokenize(lower)
    stopwords_removed=[token for token in token_list if token not in stopwords_list]
    lemma_list = [lemma.lemmatize(token) for token in stopwords_removed]
    cleaned_string = " ".join(lemma_list) #Turn the lemma list into a string for the Vectorizer
    return cleaned_string

In [30]:
#Sanity Check
clean_tweets(X_train['Tweet'].iloc[0])

'make iphone case ttye time sxsw want show support'

In [31]:
X_train['Tweet'] = X_train['Tweet'].apply(lambda x: clean_tweets(x))
X_val['Tweet'] = X_val['Tweet'].apply(lambda x: clean_tweets(x))
X_test['Tweet'] = X_test['Tweet'].apply(lambda x: clean_tweets(x))

<ipython-input-31-221299341c06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Tweet'] = X_train['Tweet'].apply(lambda x: clean_tweets(x))
<ipython-input-31-221299341c06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['Tweet'] = X_val['Tweet'].apply(lambda x: clean_tweets(x))
<ipython-input-31-221299341c06>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [32]:
#Sanity Check
X_train

,Tweet
2324,make iphone case ttye time sxsw want show support
5632,come party google tonight sxsw link band food ...
1751,winning winning gave away red mophie juice pac...
5799,google facebook official death policy vast maj...
3339,rumor google launching new social network call...
...,...
5702,even security guard austin enjoy ipad time sxs...
8604,attending sxsw want explore austin check austi...
7836,apple popup store sxsw link gonnagetanipad2
7504,putting pop apple store sxsw smart talk unders...


In [33]:
X_val

,Tweet
891,hootsuite mobile sxsw update iphone blackberry...
4198,morning hearing google circle today link sxsw
2164,great location choice nice timing ipad launch ...
1885,win ipad sxsw via sxsw link
4700,launching product sxsw plenty else join h4cker...
...,...
1033,racing around sxsw best fueling great local fa...
4186,omg still line new ipad dieing hunger sxsw els...
7735,hour sxsw popup apple store lone security guar...
8211,great app interface example moma target flipbo...


In [34]:
#Sanity Check
y_train

2324    0
5632    1
1751    0
5799    0
3339    0
       ..
5702    1
8604    0
7836    0
7504    1
3536    0
Name: Sentiment, Length: 6121, dtype: int64

In [35]:
y_val

891     0
4198    0
2164    1
1885    1
4700    0
       ..
1033    0
4186    1
7735    0
8211    1
4517    0
Name: Sentiment, Length: 2041, dtype: int64

In [36]:
#DON'T NEED BECAUSE I ADDED A LINE TO THE CLEAN_TWEETS FUNCTION

# X_train["Tweet"] = X_train["Tweet"].str.join(" ")
# X_val["Tweet"] = X_val["Tweet"].str.join(" ")
# X_test["Tweet"] = X_test["Tweet"].str.join(" ")

In [37]:
#X_train.head()

# Vectorize

In [38]:
# Import the relevant vectorizers
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [39]:
c_vectorizer = CountVectorizer()
c_vectorizer.fit(X_train['Tweet'])
X_train_c_vec = c_vectorizer.transform(X_train['Tweet'])
X_train_c_vec

<6121x7145 sparse matrix of type '<class 'numpy.int64'>'
	with 63041 stored elements in Compressed Sparse Row format>

In [40]:
X_train_c_vec_df = pd.DataFrame(X_train_c_vec.toarray(), columns=c_vectorizer.get_feature_names(), 
                              index=X_train.index)
X_train_c_vec_df

,000,0310apple,100,1000,101,106,10am,10pm,10x,10x2,...,zlf,zms,zomb,zombie,zomg,zone,zoom,zuckerberg,zynga,zzzs
2324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5632,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5799,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7836,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7504,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
#Sanity Check
X_train_c_vec_df['sxsw']

2324    1
5632    1
1751    1
5799    1
3339    1
       ..
5702    1
8604    1
7836    1
7504    1
3536    1
Name: sxsw, Length: 6121, dtype: int64

In [42]:
X_val_c_vec = c_vectorizer.transform(X_val['Tweet'])
X_val_c_vec_df = pd.DataFrame(X_val_c_vec.toarray())

In [43]:
tfidf_vectorizer = TfidfVectorizer()
    #max_df=.95,  # removes words that appear in more than 95% of docs
    #min_df=2     # removes words that appear 2 or fewer times
    #max_features=10
tfidf_vectorizer.fit(X_train['Tweet'])
X_train_tfidf_vec = tfidf_vectorizer.transform(X_train['Tweet'])
X_val_tfidf_vec = tfidf_vectorizer.transform(X_val['Tweet'])
X_train_tfidf_vec_df = pd.DataFrame(X_train_tfidf_vec.toarray())
X_val_tfidf_vec_df = pd.DataFrame(X_val_tfidf_vec.toarray())
X_train_tfidf_vec_df.shape

(6121, 7145)

# Simple Logistic Regression Model w/ Count Vectorizer

In [44]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=42)
lr.fit(X_train_c_vec_df, y_train)
print(lr.score(X_train_c_vec_df, y_train))
print(lr.score(X_val_c_vec_df, y_val))

/Users/elylin/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8921744812939062
0.7398334149926507


# Simple Logistic Regression Model w/ Tfidf Vectorizer

In [45]:
lr_2 = LogisticRegression()
lr_2.fit(X_train_tfidf_vec_df, y_train)
print(lr_2.score(X_train_tfidf_vec_df, y_train))
print(lr_2.score(X_val_tfidf_vec_df, y_val))

0.800359418395687
0.7315041646251838


# Naive Bayes w/ Count Vectorizer

In [46]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_c_vec_df, y_train)
print(naive_bayes.score(X_train_c_vec_df, y_train))
print(naive_bayes.score(X_val_c_vec_df, y_val))

0.8577029897075641
0.7158255756981872


# Naive Bayes w/ Tfidf Vectorizer

In [47]:
naive_bayes_2 = MultinomialNB()
naive_bayes_2.fit(X_train_tfidf_vec_df, y_train)
print(naive_bayes_2.score(X_train_tfidf_vec_df, y_train))
print(naive_bayes_2.score(X_val_tfidf_vec_df, y_val))

0.7892501225289985
0.7123958843704067


# Naive Bayes w/ Tuned Vectorizers

In [48]:
c_vectorizer_2 = CountVectorizer(max_df=.99,min_df=2, max_features=1000)
    #max_df=.95,  # removes words that appear in more than 95% of docs
    #min_df=2     # removes words that appear 2 or fewer times
c_vectorizer_2.fit(X_train['Tweet'])
X_train_c_vec_2 = c_vectorizer_2.transform(X_train['Tweet'])
X_val_c_vec_2 = c_vectorizer_2.transform(X_val['Tweet'])
X_train_c_vec_df_2 = pd.DataFrame(X_train_c_vec_2.toarray())
X_val_c_vec_df_2 = pd.DataFrame(X_val_c_vec_2.toarray())

In [49]:
naive_bayes_3 = MultinomialNB()
naive_bayes_3.fit(X_train_c_vec_df_2, y_train)
print("naive bayes with tuned count vectorizer")
print(naive_bayes_3.score(X_train_c_vec_df_2, y_train))
print(naive_bayes_3.score(X_val_c_vec_df_2, y_val))

naive bayes with tuned count vectorizer
0.7582094429014867
0.705046545810877


In [50]:
tfidf_vectorizer_2 = TfidfVectorizer(max_df=.99,min_df=0.005, max_features=1000)
tfidf_vectorizer_2.fit(X_train['Tweet'])
X_train_tfidf_vec_2 = tfidf_vectorizer_2.transform(X_train['Tweet'])
X_val_tfidf_vec_2 = tfidf_vectorizer_2.transform(X_val['Tweet'])
X_train_tfidf_vec_df_2 = pd.DataFrame(X_train_tfidf_vec_2.toarray())
X_val_tfidf_vec_df_2 = pd.DataFrame(X_val_tfidf_vec_2.toarray())

In [51]:
naive_bayes_4 = MultinomialNB()
naive_bayes_4.fit(X_train_tfidf_vec_df_2, y_train)
print("naive bayes with tuned tfidf")
print(naive_bayes_4.score(X_train_tfidf_vec_df_2, y_train))
print(naive_bayes_4.score(X_val_tfidf_vec_df_2, y_val))

naive bayes with tuned tfidf
0.7162228394053259
0.7011268985791279


# Logistic Regression with max iter = 1000

In [52]:
lr_3 = LogisticRegression(max_iter=1000)
lr_3.fit(X_train_c_vec_df, y_train)
print("default count vectorizer")
print(lr_3.score(X_train_c_vec_df, y_train))
print(lr_3.score(X_val_c_vec_df, y_val))

default count vectorizer
0.8921744812939062
0.739343459088682


In [53]:
#lr_3 = LogisticRegression(max_iter=1000)
lr_3.fit(X_train_tfidf_vec_df, y_train)
print("default tfidf vectorizer")
print(lr_3.score(X_train_tfidf_vec_df, y_train))
print(lr_3.score(X_val_tfidf_vec_df, y_val))

default tfidf vectorizer
0.800359418395687
0.7315041646251838


In [54]:
#lr_3 = LogisticRegression(max_iter=1000)
lr_3.fit(X_train_tfidf_vec_df_2, y_train)
print("tfidf vectorizer with tuned parameters")
print(lr_3.score(X_train_tfidf_vec_df_2, y_train))
print(lr_3.score(X_val_tfidf_vec_df_2, y_val))

tfidf vectorizer with tuned parameters
0.7288024832543702
0.7158255756981872


# PCA w/ Logistic Regression

In [55]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_c_vec_df) #Use default count vectorizer
X_val_scaled = scaler.transform(X_val_c_vec_df)

In [56]:
# Code to import, instantiate and fit a PCA object
from sklearn.decomposition import PCA

pca = PCA(n_components = .95, random_state=42)
pca.fit(X_train_scaled)
pca.n_components_

2916

In [57]:
from sklearn.pipeline import Pipeline
# Construct a pipelines
pipe_lr = Pipeline([('pca', pca), 
                    ('lr', LogisticRegression(random_state=42, max_iter=1000))])
pipe_lr.fit(X_train_scaled, y_train)
print("PCA with n_components=0.95, default count vectorizer, and logistic regression")
print(pipe_lr.score(X_train_scaled, y_train))
print(pipe_lr.score(X_val_scaled, y_val))

PCA with n_components=0.95, default count vectorizer, and logistic regression
0.9516418885802973
0.6888780009799118


In [58]:
# pipe_mnb = Pipeline([('pca', pca), 
#                     ('mnb', MultinomialNB())])
# pipe_mnb.fit(X_train_scaled, y_train)
# print("PCA with n_components=0.95, default count vectorizer, and naive bayes")
# print(pipe_lr.score(X_train_scaled, y_train))
# print(pipe_lr.score(X_val_scaled, y_val))

*I got an error about MultinomialNB not having negative values*

# PCA w/ Tuned TFIDF Vectorizer

In [59]:
scaler_2 = StandardScaler()
X_train_scaled_2 = scaler_2.fit_transform(X_train_tfidf_vec_df_2) #Use tuned tfidf vectorizer
X_val_scaled_2 = scaler_2.transform(X_val_tfidf_vec_df_2)

In [60]:
pca_2 = PCA(n_components = .90, random_state=42)
pca_2.fit(X_train_scaled_2)
pca_2.n_components_

223

In [61]:
pipe_lr_2 = Pipeline([('pca2', pca_2), 
                    ('lr2', LogisticRegression(random_state=42, max_iter=1000))])
pipe_lr_2.fit(X_train_scaled_2, y_train)
print("PCA with n_components=0.95, tuned tfidf vectorizer, and naive bayes")
print(pipe_lr_2.score(X_train_scaled_2, y_train))
print(pipe_lr_2.score(X_val_scaled_2, y_val))

PCA with n_components=0.95, tuned tfidf vectorizer, and naive bayes
0.7247181833033818
0.7143557079862812


# Pipeline and Cross Validate

In [62]:
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, plot_confusion_matrix, plot_roc_curve
from sklearn.ensemble import RandomForestClassifier

In [63]:
pipe_logreg = Pipeline(steps=[
    ('count_vectorizer', CountVectorizer()),
    ('logreg', LogisticRegression(random_state=42))
])
cv = cross_validate(pipe_logreg, X_train['Tweet'], y_train, return_train_score=True, \
                    scoring=['accuracy', 'precision','roc_auc'])
cv

/Users/elylin/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/elylin/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

{'fit_time': array([0.15934896, 0.13930917, 0.13503003, 0.13716602, 0.27472997]),
 'score_time': array([0.02159619, 0.02020407, 0.02023196, 0.03457999, 0.02587795]),
 'test_accuracy': array([0.74040816, 0.71486928, 0.75408497, 0.71078431, 0.74264706]),
 'train_accuracy': array([0.90093954, 0.90034715, 0.89810088, 0.90320604, 0.90055136]),
 'test_precision': array([0.64259928, 0.58545455, 0.68060837, 0.57      , 0.6539924 ]),
 'train_precision': array([0.93301812, 0.93019608, 0.92093023, 0.93843725, 0.93502377]),
 'test_roc_auc': array([0.75917814, 0.72217647, 0.7512428 , 0.72191419, 0.7299032 ]),
 'train_roc_auc': array([0.96578595, 0.96697679, 0.96357515, 0.96520491, 0.96583773])}

In [64]:
pipe_rfc = Pipeline(steps=[
    ('tfidf_vectorizer', TfidfVectorizer(max_df=.99,min_df=0.005, max_features=1000)),
    ('rfc', RandomForestClassifier(random_state=42))
])
cv = cross_validate(pipe_rfc, X_train['Tweet'], y_train, return_train_score=True, \
                    scoring=['accuracy', 'precision','roc_auc'])
cv

{'fit_time': array([1.72083116, 1.4621079 , 2.2294631 , 1.45498419, 1.37938499]),
 'score_time': array([0.10485101, 0.09080291, 0.09856582, 0.0896368 , 0.09111977]),
 'test_accuracy': array([0.71510204, 0.71568627, 0.71650327, 0.72058824, 0.70915033]),
 'train_accuracy': array([0.95118464, 0.94853992, 0.94894834, 0.94813151, 0.94935675]),
 'test_precision': array([0.59677419, 0.59022556, 0.59760956, 0.608     , 0.57936508]),
 'train_precision': array([0.96030116, 0.95997239, 0.96450939, 0.95676047, 0.96650384]),
 'test_roc_auc': array([0.71998169, 0.71112087, 0.70720185, 0.71120474, 0.70237856]),
 'train_roc_auc': array([0.99131925, 0.99003833, 0.99008938, 0.98925492, 0.98986803])}

# Grid Search for Random Forest Classifier

In [68]:
pg_rfc = {
    "rfc__max_depth" :[25, 50, 100],
    "rfc__min_samples_leaf" : [1, 3, 5],
    "rfc__n_estimators": [500, 1000, 1500],
    "rfc__class_weight" :['balanced'],
    "rfc__random_state":[42]
}
grid_rfc = GridSearchCV(estimator = pipe_rfc, param_grid=pg_rfc, scoring='accuracy',
                        return_train_score = True)
grid_rfc.fit(X_train['Tweet'], y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfidf_vectorizer',
                                        TfidfVectorizer(max_df=0.99,
                                                        max_features=1000,
                                                        min_df=0.005)),
                                       ('rfc',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'rfc__class_weight': ['balanced'],
                         'rfc__max_depth': [25, 50, 100],
                         'rfc__min_samples_leaf': [1, 3, 5],
                         'rfc__n_estimators': [500, 1000, 1500],
                         'rfc__random_state': [42]},
             return_train_score=True, scoring='accuracy')

In [69]:
pd.DataFrame(grid_rfc.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__class_weight,param_rfc__max_depth,param_rfc__min_samples_leaf,param_rfc__n_estimators,param_rfc__random_state,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,2.388841,0.065414,0.100762,0.001182,balanced,25,1,500,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.692369,0.008840,9,0.860703,0.865836,0.864407,0.870125,0.869308,0.866076,0.003422
1,4.817236,0.064181,0.198789,0.012140,balanced,25,1,1000,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.693513,0.007407,8,0.860090,0.862569,0.864815,0.868899,0.871554,0.865585,0.004159
2,7.189433,0.435921,0.316140,0.036888,balanced,25,1,1500,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.693513,0.008173,7,0.861315,0.864407,0.864203,0.868491,0.872779,0.866239,0.003990
3,1.669266,0.067784,0.095860,0.007585,balanced,25,3,500,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.677339,0.009797,18,0.764502,0.769655,0.759853,0.783949,0.781295,0.771851,0.009363
4,3.224799,0.089171,0.175012,0.001245,balanced,25,3,1000,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.679789,0.010623,16,0.763685,0.766592,0.760261,0.783337,0.782928,0.771361,0.009819
5,4.871765,0.153225,0.257637,0.002735,balanced,25,3,1500,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.679789,0.011646,17,0.762868,0.767409,0.760670,0.783745,0.783949,0.771728,0.010131
6,1.517088,0.036262,0.090586,0.000336,balanced,25,5,500,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.672928,0.007028,27,0.737949,0.746171,0.736778,0.762712,0.759853,0.748693,0.010815
7,3.002232,0.099937,0.172670,0.004450,balanced,25,5,1000,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.673745,0.010299,24,0.740400,0.745559,0.737799,0.763324,0.761078,0.749632,0.010586
8,4.446752,0.072476,0.253230,0.008405,balanced,25,5,1500,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.673744,0.012017,25,0.739992,0.746580,0.738615,0.763733,0.761282,0.750040,0.010558
9,4.420471,0.084525,0.138189,0.002086,balanced,50,1,500,42,"{'rfc__class_weight': 'balanced', 'rfc__max_de...",...,0.706257,0.004379,2,0.928309,0.933633,0.930570,0.926486,0.934245,0.930648,0.002988


In [70]:
grid_rfc.best_params_

{'rfc__class_weight': 'balanced',
 'rfc__max_depth': 50,
 'rfc__min_samples_leaf': 1,
 'rfc__n_estimators': 1000,
 'rfc__random_state': 42}

## CatBoost


In [66]:
from catboost import CatBoostClassifier

In [67]:
pipe_cbc = Pipeline(steps=[
    ('tfidf_vectorizer', TfidfVectorizer(max_df=.99,min_df=0.005, max_features=1000)),
    ('cbc', CatBoostClassifier())
])
cv = cross_validate(pipe_cbc, X_train['Tweet'], y_train, return_train_score=True, \
                    scoring=['accuracy', 'precision','roc_auc'])
cv

Learning rate set to 0.0203
0:	learn: 0.6902729	total: 74.7ms	remaining: 1m 14s
1:	learn: 0.6871030	total: 80.3ms	remaining: 40s
2:	learn: 0.6842272	total: 85.6ms	remaining: 28.5s
3:	learn: 0.6814182	total: 91ms	remaining: 22.7s
4:	learn: 0.6787201	total: 97ms	remaining: 19.3s
5:	learn: 0.6760546	total: 103ms	remaining: 17s
6:	learn: 0.6735657	total: 108ms	remaining: 15.3s
7:	learn: 0.6709612	total: 114ms	remaining: 14.1s
8:	learn: 0.6686637	total: 119ms	remaining: 13.1s
9:	learn: 0.6664258	total: 124ms	remaining: 12.3s
10:	learn: 0.6638814	total: 129ms	remaining: 11.6s
11:	learn: 0.6618314	total: 133ms	remaining: 11s
12:	learn: 0.6590975	total: 139ms	remaining: 10.5s
13:	learn: 0.6571945	total: 144ms	remaining: 10.1s
14:	learn: 0.6552338	total: 152ms	remaining: 9.97s
15:	learn: 0.6535929	total: 157ms	remaining: 9.65s
16:	learn: 0.6513700	total: 162ms	remaining: 9.36s
17:	learn: 0.6493916	total: 167ms	remaining: 9.09s
18:	learn: 0.6476792	total: 172ms	remaining: 8.86s
19:	learn: 0.6460

164:	learn: 0.5743828	total: 978ms	remaining: 4.95s
165:	learn: 0.5742294	total: 984ms	remaining: 4.94s
166:	learn: 0.5740444	total: 990ms	remaining: 4.94s
167:	learn: 0.5738296	total: 996ms	remaining: 4.93s
168:	learn: 0.5736574	total: 1s	remaining: 4.92s
169:	learn: 0.5733833	total: 1s	remaining: 4.91s
170:	learn: 0.5731974	total: 1.01s	remaining: 4.9s
171:	learn: 0.5730204	total: 1.02s	remaining: 4.89s
172:	learn: 0.5727775	total: 1.02s	remaining: 4.88s
173:	learn: 0.5726184	total: 1.03s	remaining: 4.87s
174:	learn: 0.5723910	total: 1.03s	remaining: 4.86s
175:	learn: 0.5721736	total: 1.04s	remaining: 4.85s
176:	learn: 0.5719787	total: 1.04s	remaining: 4.85s
177:	learn: 0.5717937	total: 1.05s	remaining: 4.84s
178:	learn: 0.5716016	total: 1.05s	remaining: 4.83s
179:	learn: 0.5713764	total: 1.06s	remaining: 4.83s
180:	learn: 0.5711611	total: 1.07s	remaining: 4.82s
181:	learn: 0.5710123	total: 1.07s	remaining: 4.81s
182:	learn: 0.5707883	total: 1.08s	remaining: 4.81s
183:	learn: 0.57055

336:	learn: 0.5470730	total: 1.97s	remaining: 3.87s
337:	learn: 0.5469322	total: 1.97s	remaining: 3.86s
338:	learn: 0.5468372	total: 1.98s	remaining: 3.86s
339:	learn: 0.5467255	total: 1.99s	remaining: 3.85s
340:	learn: 0.5465924	total: 1.99s	remaining: 3.85s
341:	learn: 0.5465017	total: 2s	remaining: 3.85s
342:	learn: 0.5463776	total: 2s	remaining: 3.84s
343:	learn: 0.5462430	total: 2.01s	remaining: 3.83s
344:	learn: 0.5461089	total: 2.02s	remaining: 3.83s
345:	learn: 0.5459840	total: 2.02s	remaining: 3.83s
346:	learn: 0.5458351	total: 2.03s	remaining: 3.82s
347:	learn: 0.5457078	total: 2.04s	remaining: 3.81s
348:	learn: 0.5456054	total: 2.04s	remaining: 3.81s
349:	learn: 0.5453975	total: 2.05s	remaining: 3.8s
350:	learn: 0.5452714	total: 2.05s	remaining: 3.8s
351:	learn: 0.5451338	total: 2.06s	remaining: 3.79s
352:	learn: 0.5450076	total: 2.07s	remaining: 3.79s
353:	learn: 0.5448855	total: 2.07s	remaining: 3.78s
354:	learn: 0.5447146	total: 2.08s	remaining: 3.78s
355:	learn: 0.544512

504:	learn: 0.5238901	total: 2.97s	remaining: 2.92s
505:	learn: 0.5235987	total: 3s	remaining: 2.92s
506:	learn: 0.5234846	total: 3s	remaining: 2.92s
507:	learn: 0.5232882	total: 3.01s	remaining: 2.92s
508:	learn: 0.5231640	total: 3.02s	remaining: 2.91s
509:	learn: 0.5230064	total: 3.02s	remaining: 2.9s
510:	learn: 0.5229138	total: 3.03s	remaining: 2.9s
511:	learn: 0.5227983	total: 3.03s	remaining: 2.89s
512:	learn: 0.5227367	total: 3.04s	remaining: 2.89s
513:	learn: 0.5225686	total: 3.05s	remaining: 2.88s
514:	learn: 0.5223934	total: 3.05s	remaining: 2.88s
515:	learn: 0.5222037	total: 3.06s	remaining: 2.87s
516:	learn: 0.5221111	total: 3.06s	remaining: 2.86s
517:	learn: 0.5219377	total: 3.07s	remaining: 2.85s
518:	learn: 0.5218442	total: 3.07s	remaining: 2.85s
519:	learn: 0.5216915	total: 3.08s	remaining: 2.84s
520:	learn: 0.5216130	total: 3.09s	remaining: 2.84s
521:	learn: 0.5214190	total: 3.09s	remaining: 2.83s
522:	learn: 0.5212689	total: 3.1s	remaining: 2.83s
523:	learn: 0.5211646

666:	learn: 0.5001889	total: 3.96s	remaining: 1.98s
667:	learn: 0.4999930	total: 3.97s	remaining: 1.97s
668:	learn: 0.4997276	total: 3.98s	remaining: 1.97s
669:	learn: 0.4995622	total: 3.98s	remaining: 1.96s
670:	learn: 0.4994092	total: 3.99s	remaining: 1.96s
671:	learn: 0.4992599	total: 4s	remaining: 1.95s
672:	learn: 0.4992176	total: 4s	remaining: 1.95s
673:	learn: 0.4991152	total: 4.01s	remaining: 1.94s
674:	learn: 0.4990553	total: 4.02s	remaining: 1.93s
675:	learn: 0.4988637	total: 4.02s	remaining: 1.93s
676:	learn: 0.4987385	total: 4.03s	remaining: 1.92s
677:	learn: 0.4986031	total: 4.03s	remaining: 1.92s
678:	learn: 0.4985016	total: 4.04s	remaining: 1.91s
679:	learn: 0.4983506	total: 4.04s	remaining: 1.9s
680:	learn: 0.4982120	total: 4.05s	remaining: 1.9s
681:	learn: 0.4980701	total: 4.06s	remaining: 1.89s
682:	learn: 0.4979386	total: 4.06s	remaining: 1.89s
683:	learn: 0.4977580	total: 4.07s	remaining: 1.88s
684:	learn: 0.4977152	total: 4.08s	remaining: 1.87s
685:	learn: 0.497610

...

369:	learn: 0.5391532	total: 1.97s	remaining: 3.35s
370:	learn: 0.5390291	total: 1.97s	remaining: 3.34s
371:	learn: 0.5388872	total: 1.98s	remaining: 3.34s
372:	learn: 0.5387474	total: 1.98s	remaining: 3.33s
373:	learn: 0.5385898	total: 1.99s	remaining: 3.33s
374:	learn: 0.5384595	total: 1.99s	remaining: 3.32s
375:	learn: 0.5383263	total: 2s	remaining: 3.32s
376:	learn: 0.5382208	total: 2s	remaining: 3.31s
377:	learn: 0.5380499	total: 2.01s	remaining: 3.31s
378:	learn: 0.5379718	total: 2.02s	remaining: 3.3s
379:	learn: 0.5378071	total: 2.02s	remaining: 3.3s
380:	learn: 0.5377114	total: 2.03s	remaining: 3.29s
381:	learn: 0.5375717	total: 2.03s	remaining: 3.29s
382:	learn: 0.5374367	total: 2.04s	remaining: 3.28s
383:	learn: 0.5373721	total: 2.04s	remaining: 3.28s
384:	learn: 0.5372298	total: 2.05s	remaining: 3.27s
385:	learn: 0.5371058	total: 2.05s	remaining: 3.27s
386:	learn: 0.5368657	total: 2.06s	remaining: 3.26s
387:	learn: 0.5366595	total: 2.06s	remaining: 3.26s
388:	learn: 0.536423

548:	learn: 0.5136982	total: 2.95s	remaining: 2.43s
549:	learn: 0.5135316	total: 2.96s	remaining: 2.42s
550:	learn: 0.5133147	total: 2.96s	remaining: 2.42s
551:	learn: 0.5132276	total: 2.97s	remaining: 2.41s
552:	learn: 0.5131259	total: 2.98s	remaining: 2.41s
553:	learn: 0.5129396	total: 2.98s	remaining: 2.4s
554:	learn: 0.5127467	total: 2.99s	remaining: 2.4s
555:	learn: 0.5126047	total: 3s	remaining: 2.39s
556:	learn: 0.5124110	total: 3s	remaining: 2.39s
557:	learn: 0.5122568	total: 3.01s	remaining: 2.38s
558:	learn: 0.5121007	total: 3.02s	remaining: 2.38s
559:	learn: 0.5120518	total: 3.02s	remaining: 2.38s
560:	learn: 0.5118808	total: 3.03s	remaining: 2.37s
561:	learn: 0.5115696	total: 3.04s	remaining: 2.37s
562:	learn: 0.5114670	total: 3.04s	remaining: 2.36s
563:	learn: 0.5112929	total: 3.05s	remaining: 2.36s
564:	learn: 0.5110562	total: 3.05s	remaining: 2.35s
565:	learn: 0.5109696	total: 3.06s	remaining: 2.35s
566:	learn: 0.5108314	total: 3.07s	remaining: 2.34s
567:	learn: 0.510751

713:	learn: 0.4906949	total: 3.93s	remaining: 1.57s
714:	learn: 0.4905955	total: 3.94s	remaining: 1.57s
715:	learn: 0.4904146	total: 3.94s	remaining: 1.56s
716:	learn: 0.4902800	total: 3.95s	remaining: 1.56s
717:	learn: 0.4901075	total: 3.95s	remaining: 1.55s
718:	learn: 0.4900613	total: 3.96s	remaining: 1.55s
719:	learn: 0.4899103	total: 3.97s	remaining: 1.54s
720:	learn: 0.4897403	total: 3.97s	remaining: 1.54s
721:	learn: 0.4896141	total: 3.98s	remaining: 1.53s
722:	learn: 0.4894683	total: 3.98s	remaining: 1.53s
723:	learn: 0.4892702	total: 3.99s	remaining: 1.52s
724:	learn: 0.4890458	total: 4s	remaining: 1.51s
725:	learn: 0.4888808	total: 4s	remaining: 1.51s
726:	learn: 0.4887065	total: 4.01s	remaining: 1.5s
727:	learn: 0.4885242	total: 4.01s	remaining: 1.5s
728:	learn: 0.4883240	total: 4.02s	remaining: 1.49s
729:	learn: 0.4882071	total: 4.02s	remaining: 1.49s
730:	learn: 0.4880374	total: 4.03s	remaining: 1.48s
731:	learn: 0.4878307	total: 4.04s	remaining: 1.48s
732:	learn: 0.487687

876:	learn: 0.4694757	total: 4.9s	remaining: 688ms
877:	learn: 0.4694458	total: 4.91s	remaining: 682ms
878:	learn: 0.4693867	total: 4.92s	remaining: 677ms
879:	learn: 0.4692150	total: 4.92s	remaining: 671ms
880:	learn: 0.4691657	total: 4.93s	remaining: 667ms
881:	learn: 0.4690114	total: 4.94s	remaining: 661ms
882:	learn: 0.4688715	total: 4.95s	remaining: 655ms
883:	learn: 0.4687446	total: 4.95s	remaining: 650ms
884:	learn: 0.4687182	total: 4.96s	remaining: 644ms
885:	learn: 0.4685848	total: 4.96s	remaining: 639ms
886:	learn: 0.4684866	total: 4.97s	remaining: 633ms
887:	learn: 0.4682836	total: 4.97s	remaining: 627ms
888:	learn: 0.4682554	total: 4.98s	remaining: 622ms
889:	learn: 0.4680381	total: 4.99s	remaining: 616ms
890:	learn: 0.4679063	total: 4.99s	remaining: 611ms
891:	learn: 0.4676606	total: 5s	remaining: 605ms
892:	learn: 0.4674881	total: 5s	remaining: 600ms
893:	learn: 0.4673504	total: 5.01s	remaining: 594ms
894:	learn: 0.4672438	total: 5.01s	remaining: 588ms
895:	learn: 0.46717

{'fit_time': array([6.25341892, 5.99257088, 5.89370108, 5.49667788, 5.95950818]),
 'score_time': array([0.03642988, 0.02947021, 0.02756095, 0.02810717, 0.03046703]),
 'test_accuracy': array([0.73632653, 0.72794118, 0.73202614, 0.71813725, 0.71078431]),
 'train_accuracy': array([0.79758987, 0.80416582, 0.79354707, 0.80437002, 0.79844803]),
 'test_precision': array([0.73717949, 0.68      , 0.72077922, 0.64912281, 0.62352941]),
 'train_precision': array([0.91468531, 0.92297297, 0.92082111, 0.92307692, 0.92112676]),
 'test_roc_auc': array([0.73420825, 0.7268061 , 0.72506771, 0.7153342 , 0.7136568 ]),
 'train_roc_auc': array([0.88700191, 0.88634704, 0.88297282, 0.88746752, 0.88956476])}